# CARLA Autopilot Demo with Third-Person Spectator View

This notebook demonstrates:
- Spawning a vehicle at a random position
- Enabling CARLA's built-in autopilot mode
- Following the vehicle with a third-person spectator camera
- Running in asynchronous mode

In [1]:
import carla
import random
import time
import math

## Configuration

In [2]:
# Connection settings
CARLA_HOST = 'localhost'
CARLA_PORT = 2000
CARLA_TIMEOUT = 20.0

# Vehicle settings
VEHICLE_FILTER = 'vehicle.*'

# Spectator settings (third-person view)
CAMERA_DISTANCE = 8.0  # Distance behind the vehicle
CAMERA_HEIGHT = 4.0    # Height above the vehicle
CAMERA_PITCH = -15.0   # Downward angle

# Simulation settings
SIMULATION_DURATION = 60.0  # Run for 60 seconds
UPDATE_FREQUENCY = 30  # Spectator updates per second

## Connect to CARLA Server

In [3]:
# Connect to CARLA
client = carla.Client(CARLA_HOST, CARLA_PORT)
client.set_timeout(CARLA_TIMEOUT)
world = client.get_world()

print(f"Connected to CARLA server")
print(f"Map: {world.get_map().name}")

Connected to CARLA server
Map: Carla/Maps/Town10HD_Opt


## Ensure Async Mode

In [4]:
# Make sure we're in async mode
settings = world.get_settings()
settings.synchronous_mode = False
settings.fixed_delta_seconds = None
world.apply_settings(settings)

print("Async mode enabled")

Async mode enabled


## Spawn Vehicle at Random Location

In [5]:
# Get available spawn points
spawn_points = world.get_map().get_spawn_points()
print(f"Found {len(spawn_points)} spawn points")

# Choose a random spawn point
spawn_point = random.choice(spawn_points)
print(f"Selected spawn point: x={spawn_point.location.x:.1f}, y={spawn_point.location.y:.1f}, z={spawn_point.location.z:.1f}")

# Get a random vehicle blueprint
blueprint_library = world.get_blueprint_library()
vehicle_blueprints = blueprint_library.filter(VEHICLE_FILTER)

# Filter for 4-wheeled vehicles (cars)
vehicle_blueprints = [bp for bp in vehicle_blueprints if int(bp.get_attribute('number_of_wheels')) == 4]

vehicle_bp = random.choice(vehicle_blueprints)
print(f"Selected vehicle: {vehicle_bp.id}")

# Set random color if available
if vehicle_bp.has_attribute('color'):
    color = random.choice(vehicle_bp.get_attribute('color').recommended_values)
    vehicle_bp.set_attribute('color', color)

# Spawn the vehicle
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
print(f"Vehicle spawned: {vehicle.type_id}")

Found 155 spawn points
Selected spawn point: x=-106.6, y=-17.1, z=0.6
Selected vehicle: vehicle.nissan.micra
Vehicle spawned: vehicle.nissan.micra


## Enable Autopilot

In [6]:
# Enable autopilot mode
vehicle.set_autopilot(True)
print("Autopilot enabled!")

Autopilot enabled!


## Third-Person Spectator View Function

In [7]:
def update_spectator_third_person(world, vehicle, distance=8.0, height=4.0, pitch=-15.0):
    """
    Update spectator camera to follow the vehicle in third-person view.
    
    Args:
        world: CARLA world object
        vehicle: The vehicle to follow
        distance: Distance behind the vehicle
        height: Height above the vehicle
        pitch: Camera pitch angle (negative = looking down)
    """
    spectator = world.get_spectator()
    vehicle_transform = vehicle.get_transform()
    
    # Get vehicle's forward vector
    yaw_rad = math.radians(vehicle_transform.rotation.yaw)
    
    # Calculate camera position behind the vehicle
    camera_x = vehicle_transform.location.x - distance * math.cos(yaw_rad)
    camera_y = vehicle_transform.location.y - distance * math.sin(yaw_rad)
    camera_z = vehicle_transform.location.z + height
    
    # Set spectator transform
    camera_transform = carla.Transform(
        carla.Location(x=camera_x, y=camera_y, z=camera_z),
        carla.Rotation(pitch=pitch, yaw=vehicle_transform.rotation.yaw, roll=0)
    )
    
    spectator.set_transform(camera_transform)

print("Third-person spectator function defined")

Third-person spectator function defined


## Run Simulation Loop

The spectator camera will follow the vehicle as it drives around using autopilot.

In [ ]:
print(f"Starting simulation for {SIMULATION_DURATION} seconds...")
print("Watch the CARLA window to see the vehicle driving!")
print("Press Ctrl+C in the terminal to stop early.")

start_time = time.time()
update_interval = 1.0 / UPDATE_FREQUENCY

try:
    while time.time() - start_time < SIMULATION_DURATION:
        # Update spectator to follow vehicle
        update_spectator_third_person(
            world, 
            vehicle,
            distance=CAMERA_DISTANCE,
            height=CAMERA_HEIGHT,
            pitch=CAMERA_PITCH
        )
        
        # Get vehicle info
        velocity = vehicle.get_velocity()
        speed_kmh = 3.6 * math.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
        location = vehicle.get_location()
        
        # Print status every second
        elapsed = time.time() - start_time
        if int(elapsed) != int(elapsed - update_interval):
            print(f"Time: {elapsed:.0f}s | Speed: {speed_kmh:.1f} km/h | Position: ({location.x:.1f}, {location.y:.1f})")
        
        time.sleep(update_interval)
        
except KeyboardInterrupt:
    print("\nSimulation interrupted by user")

print("Simulation complete!")

Starting simulation for 60.0 seconds...
Watch the CARLA window to see the vehicle driving!
Press Ctrl+C in the terminal to stop early.
Time: 1s | Speed: 0.0 km/h | Position: (-106.6, -17.1)
Time: 2s | Speed: 8.4 km/h | Position: (-106.6, -18.3)
Time: 3s | Speed: 14.8 km/h | Position: (-106.4, -21.5)
Time: 6s | Speed: 29.9 km/h | Position: (-101.3, -41.9)
Time: 8s | Speed: 26.3 km/h | Position: (-91.6, -54.1)
Time: 9s | Speed: 24.8 km/h | Position: (-85.8, -57.8)
Time: 12s | Speed: 11.7 km/h | Position: (-71.9, -61.3)
Time: 13s | Speed: 0.0 km/h | Position: (-70.8, -61.4)
Time: 14s | Speed: 0.0 km/h | Position: (-70.8, -61.4)
Time: 15s | Speed: 0.0 km/h | Position: (-70.8, -61.4)
Time: 16s | Speed: 0.0 km/h | Position: (-70.8, -61.4)
Time: 18s | Speed: 0.0 km/h | Position: (-70.8, -61.4)
Time: 19s | Speed: 0.0 km/h | Position: (-70.8, -61.4)
Time: 20s | Speed: 0.0 km/h | Position: (-70.8, -61.4)
Time: 21s | Speed: 0.0 km/h | Position: (-70.8, -61.4)
Time: 22s | Speed: 0.0 km/h | Positio

: 

## Cleanup

In [ ]:
# Destroy the vehicle
if vehicle is not None and vehicle.is_alive:
    vehicle.destroy()
    print("Vehicle destroyed")
else:
    print("Vehicle already destroyed or not found")

Vehicle destroyed
